In [42]:
from tensorflow.keras.datasets import imdb
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import *
from tensorflow.keras.preprocessing.text import Tokenizer as tk
from tensorflow.keras.callbacks import Callback
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
import numpy as np
import pandas as pd
import time

## Loading and processing the data

In [31]:
dframe = pd.read_csv('IMDB Dataset.csv')
# df = dframe.sample(20000)
df = dframe
df.head()

def Average(lst):
    return sum(lst) / len(lst)

In [5]:
df["sentiment"].value_counts()

positive    25000
negative    25000
Name: sentiment, dtype: int64

In [6]:
encoder = LabelEncoder()

df['sentiment'] = encoder.fit_transform(df['sentiment'])

In [7]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [8]:
X = np.array(df.drop(columns=['sentiment'])).reshape(len(df),)
y = np.array(df['sentiment'])

# X = np.array(df.drop(columns=['sentiment'])).reshape((-1,1))
# y = np.array(df['sentiment']).reshape((-1,1))

In [9]:
# xtr , xte , ytr, yte = train_test_split(X, y, test_size=0.15, random_state=1,stratify=y)
# xtr, xval, ytr, yval = train_test_split(xtr, ytr, test_size=0.176, random_state=1, stratify=ytr)

xtr , xte , ytr, yte = train_test_split(X, y, test_size=0.3, random_state=1,stratify=y)
xte, xval, yte, yval = train_test_split(xte, yte, test_size=0.5, random_state=1, stratify=yte)

In [11]:
tok=tk(num_words=30522, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',lower=True, split=' ',char_level=False, oov_token='Farid')

In [12]:
tok.fit_on_texts(xtr)

In [13]:
xtr1 = tok.texts_to_sequences(xtr)
xte1 = tok.texts_to_sequences(xte)
xval1 = tok.texts_to_sequences(xval)

In [14]:
sequence_len = 400
xtr2 = pad_sequences(xtr1,sequence_len, padding='post',truncating='post')
xte2 = pad_sequences(xte1,sequence_len, padding='post', truncating='post')
xval2 = pad_sequences(xval1,sequence_len, padding='post', truncating='post')

In [15]:
xtr2

array([[  12,  406,   20, ...,    0,    0,    0],
       [  12,   14,  540, ...,    0,    0,    0],
       [ 137,   11, 1342, ...,    0,    0,    0],
       ...,
       [   4,  437,  507, ...,    0,    0,    0],
       [   2,   87, 1098, ...,    0,    0,    0],
       [   2,  111,  353, ...,    0,    0,    0]], dtype=int32)

In [16]:
xtr3 = np.flip(xtr2, 1)
xte3 = np.flip(xte2, 1)
xval3 = np.flip(xval2, 1)

In [17]:
print(xtr3.shape)
print(xte3.shape)
print(xval3.shape)

(35000, 400)
(7500, 400)
(7500, 400)


## Training base model

In [ ]:
embedding_vecor_length = 768
max_review_length = 400
NUM_WORDS = 30522

tf.keras.backend.clear_session()
model = Sequential()

# input embedding vectors with CNN maxpooling
model.add(Embedding(NUM_WORDS, embedding_vecor_length, input_length=max_review_length))
model.add(Conv1D(filters=100, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D())

# LSTM (GRU) layer
model.add(GRU(50, return_sequences=True))
model.add(Dropout(0.5))

# output dense layer
model.add(Dense(100,activation='relu'))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy',metrics='accuracy')
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 400, 768)          23440896  
                                                                 
 conv1d (Conv1D)             (None, 400, 100)          230500    
                                                                 
 max_pooling1d (MaxPooling1D  (None, 200, 100)         0         
 )                                                               
                                                                 
 gru (GRU)                   (None, 200, 50)           22800     
                                                                 
 dropout (Dropout)           (None, 200, 50)           0         
                                                                 
 dense (Dense)               (None, 200, 100)          5100      
                                                        

In [16]:
class TimeForEpoch(Callback):
    def on_train_begin(self, logs={}):
        self.times=[]
        
    def on_epoch_begin(self, epoch, logs={}):
        self.starttime = time.time()
        
    def on_epoch_end(self, epoch, logs={}):
        self.times.append(time.time()-self.starttime)

cb = TimeForEpoch()

In [ ]:
history = model.fit(xtr3,ytr,epochs=2,batch_size=16,validation_data = (xval3,yval),shuffle=True,callbacks=[cb])

Epoch 1/2
2188/2188 [==============================] - 1367s 623ms/step - loss: 0.3275 - accuracy: 0.8549 - val_loss: 0.2449 - val_accuracy: 0.9096
Epoch 2/2
2188/2188 [==============================] - 1364s 623ms/step - loss: 0.1373 - accuracy: 0.9480 - val_loss: 0.2600 - val_accuracy: 0.9012


In [ ]:
def Average(lst):
    return sum(lst) / len(lst)

average_epoch_time = Average(cb.times)
print("Average epoch time:", average_epoch_time)

Average epoch time: 1365.2078201770782


In [ ]:
starttime = time.time()
print("Test accuracy: ", np.round(model.evaluate(xte3,yte,verbose=0)[1]*100,3),"%")
endtime = time.time() - starttime

Test accuracy:  90.347 %


In [ ]:
starttime = time.time()
print("Validation accuracy: ", np.round(model.evaluate(xval3,yval,verbose=0)[1]*100,3),"%")
endtime = time.time() - starttime

Validation accuracy:  90.12 %


In [ ]:
print("Average time per prediction:", endtime/len(xte3))

Average time per prediction: 0.005036624336242676


## Tuning number of hidden neurons

### 100 neurons in Dense layer

In [44]:
def make_model(neurons):
  embedding_vecor_length = 768
  max_review_length = 400
  NUM_WORDS = 30522

  tf.keras.backend.clear_session()
  model = Sequential()
  model.add(Embedding(NUM_WORDS, embedding_vecor_length, input_length=max_review_length))
  model.add(Conv1D(filters=100, kernel_size=3, padding='same', activation='relu'))
  model.add(MaxPooling1D())
  model.add(GRU(neurons, return_sequences=True))
  # model.add(GRU(50))
  model.add(Dropout(0.5))
  model.add(Dense(100,activation='relu'))
  model.add(Flatten())
  model.add(Dense(1, activation='sigmoid'))
  return model

In [45]:
model_100_neurons = make_model(100)
model_100_neurons.compile(optimizer='adam', loss='binary_crossentropy',metrics='accuracy')
model_100_neurons.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 400, 768)          23440896  
                                                                 
 conv1d (Conv1D)             (None, 400, 100)          230500    
                                                                 
 max_pooling1d (MaxPooling1D  (None, 200, 100)         0         
 )                                                               
                                                                 
 gru (GRU)                   (None, 100)               60600     
                                                                 
 dropout (Dropout)           (None, 100)               0         
                                                                 
 dense (Dense)               (None, 100)               10100     
                                                        

In [ ]:
early_stopping1 = EarlyStopping(monitor='val_loss', patience = 2, mode = 'min')
history = model_100_neurons.fit(xtr3,ytr,epochs=2,batch_size=16,validation_data = (xval3,yval),shuffle=True,callbacks=[cb, early_stopping1])

In [1]:
def Average(lst):
    return sum(lst) / len(lst)
    
average_epoch_time = Average(cb.times)
print("Average epoch time:", average_epoch_time)

Average epoch time: 1584.593629118219


In [48]:
starttime = time.time()
print("Test accuracy: ", np.round(model_100_neurons.evaluate(xte3,yte,verbose=0)[1]*100,3),"%")
endtime = time.time() - starttime

Test accuracy:  90.573 %


In [49]:
starttime = time.time()
print("Validation accuracy: ", np.round(model_100_neurons.evaluate(xval3,yval,verbose=0)[1]*100,3),"%")
endtime = time.time() - starttime

Validation accuracy:  89.987 %


In [50]:
print("Average time per prediction:", endtime/len(xte3))

Average time per prediction: 0.0005742523193359375


### 200 neurons in Dense layer

In [ ]:
model_200_neurons = make_model(200)
model_200_neurons.compile(optimizer='adam', loss='binary_crossentropy',metrics='accuracy')
model_200_neurons.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 400, 768)          23440896  
                                                                 
 conv1d (Conv1D)             (None, 400, 100)          230500    
                                                                 
 max_pooling1d (MaxPooling1D  (None, 200, 100)         0         
 )                                                               
                                                                 
 gru (GRU)                   (None, 200, 200)          181200    
                                                                 
 dropout (Dropout)           (None, 200, 200)          0         
                                                                 
 dense (Dense)               (None, 200, 100)          20100     
                                                        

In [ ]:
history = model_200_neurons.fit(xtr3,ytr,epochs=2,batch_size=16,validation_data = (xval3,yval),shuffle=True,callbacks=[cb])

In [2]:
average_epoch_time = Average(cb.times)
print("Average epoch time:", average_epoch_time)

Average epoch time: 2148.859926836297


In [ ]:
starttime = time.time()
print("Test accuracy: ", np.round(model_200_neurons.evaluate(xte3,yte,verbose=0)[1]*100,3),"%")
endtime = time.time() - starttime

Test accuracy:  90.053 %


In [ ]:
starttime = time.time()
print("Validation accuracy: ", np.round(model_200_neurons.evaluate(xval3,yval,verbose=0)[1]*100,3),"%")
endtime = time.time() - starttime

Validation accuracy:  90.4 %


In [ ]:
print("Average time per prediction:", endtime/len(xte3))

Average time per prediction: 0.0006709656079610189


## Tuning number of layers

### 2 hidden layers

In [ ]:
def make_layer_model(layer):
  embedding_vecor_length = 768
  max_review_length = 400
  NUM_WORDS = 30522

  tf.keras.backend.clear_session()
  model = Sequential()
  model.add(Embedding(NUM_WORDS, embedding_vecor_length, input_length=max_review_length))
  model.add(Conv1D(filters=100, kernel_size=3, padding='same', activation='relu'))
  model.add(MaxPooling1D())
  model.add(GRU(100, return_sequences=True))
  for i in range(layer):
    model.add(GRU(50))
  model.add(Dropout(0.5))
  # for i in range(layer):
  model.add(Dense(100,activation='relu'))
  model.add(Flatten())
  model.add(Dense(1, activation='sigmoid'))
  return model

In [ ]:
model_2_layers = make_layer_model(1)
model_2_layers.compile(optimizer='adam', loss='binary_crossentropy',metrics='accuracy')
model_2_layers.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 400, 768)          23440896  
                                                                 
 conv1d (Conv1D)             (None, 400, 100)          230500    
                                                                 
 max_pooling1d (MaxPooling1D  (None, 200, 100)         0         
 )                                                               
                                                                 
 gru (GRU)                   (None, 200, 100)          60600     
                                                                 
 gru_1 (GRU)                 (None, 50)                22800     
                                                                 
 dropout (Dropout)           (None, 50)                0         
                                                        

In [ ]:
history = model_2_layers.fit(xtr3,ytr,epochs=2,batch_size=16,validation_data = (xval3,yval),shuffle=True,callbacks=[cb])

In [3]:
average_epoch_time = Average(cb.times)
print("Average epoch time:", average_epoch_time)

Average epoch time: 1308.1409307718277


In [ ]:
starttime = time.time()
print("Test accuracy: ", np.round(model_2_layers.evaluate(xte3,yte,verbose=0)[1]*100,3),"%")
endtime = time.time() - starttime

Test accuracy:  89.333 %


In [ ]:
starttime = time.time()
print("Validation accuracy: ", np.round(model_2_layers.evaluate(xval3,yval,verbose=0)[1]*100,3),"%")
endtime = time.time() - starttime

Validation accuracy:  89.373 %


In [ ]:
print("Average time per prediction:", endtime/len(xte3))

Average time per prediction: 0.0010443187395731608


## Keras LSTM with attention

In [18]:
# Add attention layer to the deep learning network
class attention(Layer):
    def __init__(self,**kwargs):
        super(attention,self).__init__(**kwargs)
 
    def build(self,input_shape):
        self.W=self.add_weight(name='attention_weight', shape=(input_shape[-1],1), 
                               initializer='random_normal', trainable=True)
        self.b=self.add_weight(name='attention_bias', shape=(input_shape[1],1), 
                               initializer='zeros', trainable=True)        
        super(attention, self).build(input_shape)
 
    def call(self,x):
        # Alignment scores. Pass them through tanh function
        e = K.tanh(K.dot(x,self.W)+self.b)
        # Remove dimension of size 1
        e = K.squeeze(e, axis=-1)   
        # Compute the weights
        alpha = K.softmax(e)
        # Reshape to tensorFlow format
        alpha = K.expand_dims(alpha, axis=-1)
        # Compute the context vector
        context = x * alpha
        context = K.sum(context, axis=1)
        return context

In [51]:
def myAct(out):
    return K.softmax(K.tanh(out))

embedding_vecor_length = 768
max_review_length = 400
NUM_WORDS = 30522

tf.keras.backend.clear_session()
model = Sequential()
model.add(Embedding(NUM_WORDS, embedding_vecor_length, input_length=max_review_length))
model.add(Conv1D(filters=100, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D())
model.add(GRU(100, return_sequences=False, activation=myAct))
# model.add(GRU(50))
model.add(Dropout(0.5))
model.add(Dense(100,activation='relu'))

# model.add(attention())
# model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

In [52]:
model.compile(optimizer='adam', loss='binary_crossentropy',metrics='accuracy')
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 400, 768)          23440896  
                                                                 
 conv1d (Conv1D)             (None, 400, 100)          230500    
                                                                 
 max_pooling1d (MaxPooling1D  (None, 200, 100)         0         
 )                                                               
                                                                 
 gru (GRU)                   (None, 100)               60600     
                                                                 
 dropout (Dropout)           (None, 100)               0         
                                                                 
 dense (Dense)               (None, 100)               10100     
                                                        

In [53]:
class TimeForEpoch(Callback):
    def on_train_begin(self, logs={}):
        self.times=[]
        
    def on_epoch_begin(self, epoch, logs={}):
        self.starttime = time.time()
        
    def on_epoch_end(self, epoch, logs={}):
        self.times.append(time.time()-self.starttime)

cb = TimeForEpoch()

In [5]:
early_stopping1 = EarlyStopping(monitor='val_loss', patience = 2, mode = 'min')
history = model.fit(xtr3,ytr,epochs=2,batch_size=16,validation_data = (xval3,yval),shuffle=True,callbacks=[cb, early_stopping1])

In [4]:
def Average(lst):
    return sum(lst) / len(lst)

average_epoch_time = Average(cb.times)
print("Average epoch time:", average_epoch_time)

Average epoch time: 1038.546227812767


In [ ]:
starttime = time.time()
print("Accuracy: ", np.round(model.evaluate(xte3,yte,verbose=0)[1]*100,3),"%")
endtime = time.time() - starttime

Accuracy:  88.987 %


In [ ]:
starttime = time.time()
print("Validation accuracy: ", np.round(model.evaluate(xval3,yval,verbose=0)[1]*100,3),"%")
endtime = time.time() - starttime

Validation accuracy:  89.067 %


In [ ]:
print("Average time per prediction:", endtime/len(xte3))

Average time per prediction: 0.0019438776969909668
